In [62]:
import json
dictionary={}
hateful_or_not={}
count=0
with open ("../data/train.jsonl",'r') as f:
    for line in f:
        elem=json.loads(line)
        if (int(elem['label'])==0):
            hateful_or_not[int(elem['id'])]=0
        else:
            hateful_or_not[int(elem['id'])]=1
            count+=1
# print(count)
probab_hateful=count/len(hateful_or_not)
words_object_hateful={}
words_object_normal={}
with open("object_details.json", 'r') as f:
    dictionary=json.load(f)
    for (image_id,elem) in dictionary.items():
        # print(sentence)
        for inndexing,id in enumerate(elem['class_ids']):
            # if (id==11):
                # print(image_id)
            if (hateful_or_not[int(image_id)]==0):
                if id in words_object_normal:
                    words_object_normal[id]+=elem['confs'][inndexing]
                else:
                    words_object_normal[id]=elem['confs'][inndexing]
            else:
                if id in words_object_hateful:
                    words_object_hateful[id]+=elem['confs'][inndexing]
                else:
                    words_object_hateful[id]=elem['confs'][inndexing]


print(words_object_hateful)
print(words_object_normal)
with open("words_object_hateful.json", 'w') as f:
    json.dump(words_object_hateful,f)
with open("words_object_normal.json", 'w') as f:
    json.dump(words_object_normal,f)

{0: 4747.74540501833, 27: 293.82346946001053, 16: 159.41741746664047, 43: 7.436751425266266, 53: 7.248833656311035, 45: 31.745578587055206, 60: 27.39535015821457, 42: 5.0009805560112, 44: 14.065178573131561, 58: 19.999642312526703, 41: 72.29670107364655, 26: 25.16674542427063, 25: 8.601253390312195, 34: 19.734644770622253, 18: 62.659197211265564, 67: 66.31527000665665, 56: 109.67237162590027, 5: 5.4802815318107605, 17: 18.092341005802155, 21: 21.076729476451874, 24: 24.661790311336517, 57: 29.105053544044495, 8: 13.188944399356842, 39: 52.230327010154724, 40: 12.590883135795593, 79: 11.952634453773499, 3: 9.803335130214691, 38: 2.6151154041290283, 2: 101.28770381212234, 28: 3.8886606693267822, 63: 33.4433159828186, 15: 32.78496390581131, 54: 5.734434962272644, 73: 38.39191633462906, 71: 7.049708664417267, 76: 6.135054588317871, 77: 12.465758919715881, 59: 33.958972811698914, 13: 18.2206472158432, 19: 12.149249970912933, 62: 9.163593411445618, 7: 27.408604562282562, 55: 3.83325153589248

In [128]:
probab_hateful=1/2
import numpy as np

class NaiveBayesClassifier:
    def __init__(self):
        self.class_prior = {}
        self.word_likelihoods = {}

    def train(self, word_counts, hateful_dict, normal_dict):
        # for val in hatef
        total_hateful=0
        total_normal=0
        for val in hateful_dict.values():
            total_hateful+=val
        for val in normal_dict.values():
            total_normal+=val
        # total_hateful = sum(hateful_counts)
        # total_normal = sum(normal_counts)
        total_examples = total_hateful + total_normal
        
        # self.class_prior[1] = total_hateful / total_examples
        # self.class_prior[0] = total_normal / total_examples
        self.class_prior[1] = probab_hateful
        self.class_prior[0] = 1 - probab_hateful
        
        for word in word_counts:
            hateful_count=0
            normal_count=0
            if (word in hateful_dict):
                hateful_count=int(hateful_dict[word])
            if (word in normal_dict):
                normal_count=int(normal_dict[word])
            prob_hateful = (hateful_count + 1) / (total_hateful + len(word_counts))
            prob_normal = (normal_count + 1) / (total_normal + len(word_counts))
            self.word_likelihoods[word] = (prob_hateful, prob_normal)

    def predict(self, X_test):
        predictions = []
        for text in X_test:
            prob_hateful = np.log(0.9996560693296879*self.class_prior[1])
            prob_normal = np.log(self.class_prior[0])
            for word, count in text.items():
                if word in self.word_likelihoods:
                    prob_hateful += count * np.log(self.word_likelihoods[word][0])
                    prob_normal += count * np.log(self.word_likelihoods[word][1])
                    # print(prob_hateful)
                    # print(prob_normal)
            predictions.append(1 if prob_hateful > prob_normal else 0)
        return predictions

# Example usage:
# print(prob_hateful)
# word_counts = ["love", "hate", "awesome", "stupid"]
# hateful_counts = [1, 3, 0, 1]
# normal_counts = [2, 0, 1, 2]
word_counts=[]
for word,count in words_object_hateful.items():
    word_counts.append(word)
for word,count in words_object_normal.items():
    if word not in word_counts:
        word_counts.append(word)


classifier = NaiveBayesClassifier()
classifier.train(word_counts,words_object_hateful, words_object_normal)

# X_test = [{11: 5}, {1: 1, 2: 1}]
# predictions = classifier.predict(X_test)
# print(predictions)  # Output: [1, 0] indicating hateful and non-hateful respectively


In [129]:
import numpy as np

class GaussianNaiveBayesClassifier:
    def __init__(self):
        self.class_prior = {}
        self.word_stats = {}

    def train(self, word_counts, hateful_dict, normal_dict):
        total_hateful=0
        total_normal=0
        for val in hateful_dict.values():
            total_hateful+=val
        for val in normal_dict.values():
            total_normal+=val
        self.class_prior[1] = probab_hateful
        self.class_prior[0] = 1 - probab_hateful
        
        # self.class_prior[1] = total_hateful / (total_hateful+total_normal)
        # self.class_prior[0] = total_normal / (total_hateful+total_normal)
        
        for word in word_counts:
            hateful_count=0
            normal_count=0
            if (word in hateful_dict):
                hateful_count=int(hateful_dict[word])
            if (word in normal_dict):
                normal_count=int(normal_dict[word])
            mean_hateful = hateful_count / total_hateful
            mean_normal = normal_count / total_normal
            variance_hateful = (hateful_count * (1 - mean_hateful)**2 + (total_hateful - hateful_count) * (0 - mean_hateful)**2) / total_hateful
            variance_normal = (normal_count * (1 - mean_normal)**2 + (total_normal - normal_count) * (0 - mean_normal)**2) / total_normal
            self.word_stats[word] = (mean_hateful, variance_hateful, mean_normal, variance_normal)

    def predict(self, X_test):
        predictions = []
        for text in X_test:
            prob_hateful = np.log(self.class_prior[1])
            prob_normal = np.log(self.class_prior[0])
            for word, count in text.items():
                if word in self.word_stats:
                    mean_hateful, variance_hateful, mean_normal, variance_normal = self.word_stats[word]
                    prob_hateful += self.gaussian_prob(count, mean_hateful, variance_hateful)
                    prob_normal += self.gaussian_prob(count, mean_normal, variance_normal)
            predictions.append(1 if prob_hateful > prob_normal else 0)
        return predictions


    def gaussian_prob(self, x, mean, variance):
        exponent = -((x - mean) ** 2) / (2 * (variance+1e-9))
        return np.exp(exponent) / np.sqrt(2 * np.pi * (variance+1e-9))

# Example usage:
# word_counts = ["love", "hate", "awesome", "stupid"]
# hateful_counts = [1, 3, 0, 1]
# normal_counts = [2, 0, 1, 2]

classifier = NaiveBayesClassifier()
classifier.train(word_counts, words_object_hateful, words_object_normal)

# X_test = [{11: 0}, { 1:0}]
with open ("object_details_test.json") as f:
    object_details_test=json.load(f)
X_test=[]
answers=[]
for image_id,elem in object_details_test.items():
    for index,id in enumerate(elem['class_ids']):
        temp_dict={id:elem['confs'][index]}
    X_test.append(temp_dict)
    answers.append(elem['label'])

# from decimal import Decimal

# low = Decimal('0.7')
# high = Decimal('1.5')
# precision = Decimal('1e-6')

# while high - low >= precision:
#     mid1 = low + (high - low) / Decimal('3')
#     mid2 = high - (high - low) / Decimal('3')
    
#     predictions1 = classifier.predict(X_test, float(mid1))
#     predictions2 = classifier.predict(X_test, float(mid2))
    
#     count1 = sum(1 for answer, predict in zip(answers, predictions1) if answer == predict)
#     count2 = sum(1 for answer, predict in zip(answers, predictions2) if answer == predict)
    
#     if count1 > count2:
#         high = mid2
#     else:
#         low = mid1

# best_factor = (low + high) / Decimal('2')

# print("Best factor:", float(best_factor))
# print("Max count:", max(count1, count2))

# low = 0.7
# high = 1.3
# max_count = 0
# best_factor = low

# while high - low >= 1e-6:
#     mid = (low + high) / 2
#     predictions = classifier.predict(X_test, mid)
#     count = 0
#     count2 = 0
#     for answer, predict in zip(answers, predictions):
#         if answer == predict:
#             if answer == 0:
#                 count += 1
#             else:
#                 count2 += 1
#     total_count = count + count2
#     if total_count > max_count:
#         max_count = total_count
#         best_factor = mid
#     if count > count2:
#         low = mid
#     else:
#         high = mid

# print("Best factor:", best_factor)
# print("Max count:", max_count)


predictions = classifier.predict(X_test)
print(predictions)  # Output: [1, 0] indicating hateful and non-hateful respectively
count=0
count2=0
for answer,predict in zip(answers,predictions):
    if (answer==predict):
        if (answer==0):
            count+=1
        else:
            count2+=1
        # print("Haash")
        # count+=1
print(count,count2)


[1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 